In [1]:
import pandas as pd

In [2]:
# Load and process the data
file_path = './project/cluster_usage_data.txt'
df = pd.read_csv(file_path, delimiter='|')

Adding column of totalCpuTime for each job by seconds and converting start and end dates

In [3]:
def parse_time(time_str):
    parts = time_str.split('-')
    if len(parts) == 2:
        days, time = parts
    else:
        days = 0
        time = parts[0]

    hours, minutes, seconds = map(int, time.split(':'))
    total_seconds = int(days) * 86400 + hours * 3600 + minutes * 60 + seconds
    return total_seconds


df["CPUTimeSeconds"] = df["CPUTime"].apply(parse_time)

df['Start'] = pd.to_datetime(df['Start'], errors='coerce', utc=True)
df['End'] = pd.to_datetime(df['End'], errors='coerce', utc=True)


df.head()
print(df.columns)

Index(['JobID', 'User', 'Account', 'Partition', 'State', 'AllocCPUS',
       'Elapsed', 'Start', 'End', 'NCPUS', 'NNodes', 'TotalCPU', 'UserCPU',
       'SystemCPU', 'CPUTime', 'CPUTimeSeconds'],
      dtype='object')


C:\Users\farid\AppData\Local\Temp\ipykernel_12748\1234020961.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['End'] = pd.to_datetime(df['End'], errors='coerce', utc=True)


# Processing by User

In [4]:
def getTotalTimeAll():
    temp = df.groupby('User')['CPUTimeSeconds'].sum()
    return temp


# example usage
getTotalTimeAll()


User
abdelmoujoud.faris            0
amine.andam              661891
badr-eddine.semlali      983038
bernardin.ligan         6912680
haitham.naciri          7978356
hamsbaai               12619640
ikissami                  33187
ilyas.bouziani          1956020
issam.aityahia           227340
noureddine.toutlini      617230
olivier.djara           6912280
safae                   1590771
said.grich                61927
sboughou                 721865
slahbabi               15387060
swiam                         0
team1337                 129360
Name: CPUTimeSeconds, dtype: int64

In [5]:
def getTotalTimeUser(user):
    return getTotalTimeAll()[user]

#example usage:
getTotalTimeUser('safae')


1590771

Cpu time per Partition

In [6]:

#Grouping by the type partitions
def CpuTimePartitionAll():
    return df.groupby("Partition")["CPUTimeSeconds"].sum()

def CpuTimePartition(partition):
    return CpuTimePartitionAll()[partition]


CpuTimePartitionAll()

Partition
defq           1574
gpu        28567697
longq      27741792
shortq       307728
special        1021
visu         172833
Name: CPUTimeSeconds, dtype: int64

In [18]:
df.groupby(["Partition", "User"])["CPUTimeSeconds"].sum()
# df.head()

Partition  User               
defq       ikissami                   1574
gpu        abdelmoujoud.faris            0
           amine.andam              661891
           bernardin.ligan         6912680
           hamsbaai               12619640
           ikissami                  30592
           ilyas.bouziani           215380
           noureddine.toutlini      617230
           olivier.djara           6912280
           sboughou                 468644
           swiam                         0
           team1337                 129360
longq      badr-eddine.semlali      983038
           haitham.naciri          7978356
           ilyas.bouziani          1740640
           safae                   1590771
           said.grich                61927
           slahbabi               15387060
shortq     issam.aityahia           227340
           sboughou                  80388
special    ikissami                   1021
visu       sboughou                 172833
Name: CPUTimeSeconds, d

Cpu Time per Account

In [7]:
def TotalTimeAccountAll():
    return df.groupby("Account")["CPUTimeSeconds"].sum()

def TotalTimeAccount(account):
    return TotalTimeAccountAll()[account]


TotalTimeAccountAll()

Account
agbs-account            1234460
ak-account             13825440
crsa-account            1106893
d4r-account            12620740
limset-account          3912436
msda-account           48025570
novec-account            182183
sccs-dna-account        1551128
usmba-limas-account           0
Name: CPUTimeSeconds, dtype: int64

Filtering by start and end Date

In [8]:
def filter_jobs_within_dates(start_date, end_date):
    # Convert input dates to UTC timezone-aware datetime objects
    start_date_utc = pd.to_datetime(start_date).tz_localize('UTC')
    end_date_utc = pd.to_datetime(end_date).tz_localize('UTC')

    # Filter the DataFrame for jobs within the specified date range
    # Both Start and End dates must be within the range
    filtered_df = df[(df['Start'] >= start_date_utc) & (df['End'].fillna(pd.Timestamp('now').tz_localize('UTC')) <= end_date_utc)]
    return filtered_df


# Example usage
start_date = '2023-12-08T00:00:00'
end_date = '2023-12-12T23:59:59'

filtered_jobs = filter_jobs_within_dates(start_date, end_date)
print(filtered_jobs)



             JobID                 User           Account Partition  \
2          5858569             slahbabi      msda-account     longq   
3    5858569.batch                  NaN      msda-account       NaN   
4          5858619          amine.andam  sccs-dna-account       gpu   
5    5858619.batch                  NaN  sccs-dna-account       NaN   
6          5858668      bernardin.ligan        ak-account       gpu   
..             ...                  ...               ...       ...   
241  5858899.batch                  NaN    limset-account       NaN   
242        5858900                safae      msda-account     longq   
243  5858900.batch                  NaN      msda-account       NaN   
244        5858901             sboughou      msda-account    shortq   
247        5858904  badr-eddine.semlali      crsa-account     longq   

                 State  AllocCPUS     Elapsed                     Start  \
2              TIMEOUT         30  3-00:00:02 2023-12-09 17:56:07+00:00 

Group by state

In [9]:
def jobsState():
    return df.groupby("State").size()

jobsState()



State
CANCELLED             27
CANCELLED by 0         1
CANCELLED by 1045      6
CANCELLED by 1063     14
CANCELLED by 1065      1
CANCELLED by 1188      5
CANCELLED by 1437      9
CANCELLED by 1489      1
CANCELLED by 1496      4
COMPLETED             58
FAILED               102
OUT_OF_MEMORY          1
PENDING                5
RUNNING                5
TIMEOUT               13
dtype: int64

TotalCpuTime per day for each user for a given period of time

In [10]:
def calculate_daily_cpu_times(row):
    start = row['Start']
    end = row['End'] if pd.notna(row['End']) else pd.Timestamp('now').tz_localize('UTC')
    daily_times = {}

    current = start
    while current.date() <= end.date():
        next_day = (current + pd.Timedelta(days=1)).replace(hour=0, minute=0, second=0)
        daily_end = min(end, next_day)
        seconds = (daily_end - current).total_seconds()
        date_str = f"{current.date()} {current.strftime('%A')}"
        daily_times[date_str] = seconds * row['AllocCPUS']
        current = next_day

    return pd.Series(daily_times)


def aggregate_daily_cpu_times(df, column):
    daily_data = df.apply(calculate_daily_cpu_times, axis=1)
    daily_data[column] = df[column]
    return daily_data.melt(id_vars=[column], var_name='Date', value_name='CPUTimeSeconds').groupby([column, 'Date']).sum()

# wa7d l exemple
start_date = '2023-12-12T00:00:00'
end_date = '2023-12-13T23:59:59'


filtered_df = filter_jobs_within_dates( start_date, end_date)
daily_cpu_times = aggregate_daily_cpu_times(filtered_df, 'User')
print(daily_cpu_times)

                                          CPUTimeSeconds
User                Date                                
amine.andam         2023-12-12 Tuesday          113927.0
                    2023-12-13 Wednesday         98323.0
badr-eddine.semlali 2023-12-12 Tuesday          560602.0
                    2023-12-13 Wednesday        422436.0
hamsbaai            2023-12-12 Tuesday            1056.0
                    2023-12-13 Wednesday             0.0
ikissami            2023-12-12 Tuesday           33187.0
                    2023-12-13 Wednesday             0.0
ilyas.bouziani      2023-12-12 Tuesday         1956020.0
                    2023-12-13 Wednesday             0.0
issam.aityahia      2023-12-12 Tuesday          227340.0
                    2023-12-13 Wednesday             0.0
safae               2023-12-12 Tuesday           24660.0
                    2023-12-13 Wednesday             0.0
said.grich          2023-12-12 Tuesday           19740.0
                    2023-12-13 

In [11]:
filtered_df = filter_jobs_within_dates(start_date, end_date)
daily_cpu_times = aggregate_daily_cpu_times(filtered_df, 'Account')
print(daily_cpu_times)

                                       CPUTimeSeconds
Account          Date                                
crsa-account     2023-12-12 Tuesday          600082.0
                 2023-12-13 Wednesday        506811.0
d4r-account      2023-12-12 Tuesday            2156.0
                 2023-12-13 Wednesday             0.0
limset-account   2023-12-12 Tuesday         3912436.0
                 2023-12-13 Wednesday             0.0
msda-account     2023-12-12 Tuesday         1223064.0
                 2023-12-13 Wednesday         97162.0
novec-account    2023-12-12 Tuesday          182183.0
                 2023-12-13 Wednesday             0.0
sccs-dna-account 2023-12-12 Tuesday          455194.0
                 2023-12-13 Wednesday        196648.0


In [12]:
aggregate_daily_cpu_times(filtered_df, 'Partition')

CPUTimeSeconds
Partition Date                                
defq      2023-12-12 Tuesday            1574.0
          2023-12-13 Wednesday             0.0
gpu       2023-12-12 Tuesday          958959.0
          2023-12-13 Wednesday         98323.0
longq     2023-12-12 Tuesday         2345642.0
          2023-12-13 Wednesday        464623.0
shortq    2023-12-12 Tuesday          307728.0
          2023-12-13 Wednesday             0.0
special   2023-12-12 Tuesday            1021.0
          2023-12-13 Wednesday             0.0
visu      2023-12-12 Tuesday           37840.0
          2023-12-13 Wednesday         48580.0

Top users according to TotalCpuTime within a period of time

In [13]:
def topUsers(startDate, endDate):
    return filter_jobs_within_dates(startDate, endDate).groupby('User')['CPUTimeSeconds'].sum().sort_values(ascending=False)

In [14]:
topUsers(start_date, end_date)

User
ilyas.bouziani         1956020
badr-eddine.semlali     983038
sboughou                635452
issam.aityahia          227340
amine.andam             212250
team1337                129360
said.grich               61927
ikissami                 33187
safae                    24660
hamsbaai                  1056
swiam                        0
Name: CPUTimeSeconds, dtype: int64